In [14]:
import warnings
warnings.filterwarnings('ignore')
import torch
import os
from pprint import pprint
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
# from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
# Import things that are needed generically
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool

from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai import ChatOpenAI

from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent

In [2]:
import semantic_scholoar_api as ss

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /c1/yumin/.cache/huggingface/token
Login successful


In [3]:
dotenv_path = '../../../.env'
load_dotenv(dotenv_path)
# openai_api = os.getenv("OPENAI_API_KEY")
api_key = os.getenv("SEMANTIC_SCHOLAR_API_KEY")


### Define Tools

In [5]:
class load_paper_input(BaseModel):
    query: str = Field(description="target paper title")
    start_page: int = Field(description='start page number')
    
loadpaper = StructuredTool.from_function(
    func=ss.query2content,
    name="loadpaper",
    description="The `loadPaper` tool is designed to facilitate the process of retrieving and reading academic papers based on a given search query. The `query` parameter is a string representing the title of the paper. The `loadPaper` tool extracts and returns the text content from the PDF. If the paper does not contain the desired information, you can adjust the start_page argument to specify the starting page for reading. For instance, to extract the title, abstract, or detailed method sections, set start_page to 1. Conversely, for sections like the appendix that are typically found towards the end of the paper, set start_page to a higher number, such as 9.",
    args_schema=load_paper_input
)

In [6]:
class recommend_reference_input(BaseModel):
    query: str = Field(description="target paper title")

recommend_reference = StructuredTool.from_function(
    func=ss.reference_recommend,
    name="recommend_reference",
    description="The reference_recommend function recommends relevant academic papers based on a given query, focusing on papers that the target paper's references. This tool is ideal for researchers and academics looking to find related literature that has been directly cited by the target paper.",
    args_schema=recommend_reference_input
)

In [7]:
class citation_recommend_input(BaseModel):
    query: str = Field(description="target paper title")

recommend_citation = StructuredTool.from_function(
    func=ss.citation_recommend,
    name="recommend_citation",
    description="The recommend_citation function identifies and recommends subsequent papers **that have cited a given target paper**, providing valuable insights into the evolution and impact of the research. This tool helps researchers discover influential follow-up studies and stay updated with the latest developments in their field.",
    args_schema=citation_recommend_input
)

### Make Agent with GPT3.5

In [11]:
model = ChatOpenAI(model="gpt-3.5-turbo")

In [12]:
tools = [loadpaper, recommend_reference, recommend_citation]
# functions = [convert_to_openai_function(t) for t in tools]
# functions[1]

In [15]:
# load Agent prompt
prompt = hub.pull("hwchase17/openai-tools-agent")

In [16]:
# Choose the LLM that will drive the agent
# Only certain models support this
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Construct the OpenAI Tools agent
agent = create_openai_tools_agent(llm, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [30]:
# If you want the model to see the specific page or specific content in the paper, mention the page in the prompt.

output = agent_executor.invoke({"input": "explain about the math expression about the Filtering api calls in the 3 page of the paper 'Language Models Can Teach Themselves to Use Tools'"})



> Entering new AgentExecutor chain...

Invoking: `loadpaper` with `{'query': 'Language Models Can Teach Themselves to Use Tools', 'start_page': 3}`


### Searched Paper ###
Title: Toolformer: Language Models Can Teach Themselves to Use Tools
Authors: Thomas Scialom
Published: 2023-02-09T16:49:57Z
ID: 2302.04761v1
File ./papers_db/2302.04761v1.pdf already exists. Skipping download.
Page limit reached at 4
Your task is to add calls to a Question Answering API to a piece of text. The questions should help you get information required to complete the text. You can call the API by writing "[QA(question)]" where "question" is the question you want to ask. Here are some examples of API calls: Input: Joe Biden was born in Scranton, Pennsylvania. Output: Joe Biden was born in [QA("Where was Joe Biden born?")] Scranton, [QA("In which state is Scranton?")] Pennsylvania. Input: Coca-Cola, or Coke, is a carbonated soft drink manufactured by the Coca-Cola Company. Output: Coca-Cola, or [QA("What o

In [31]:
pprint(output['output'])

('The math expression about the filtering of API calls in the paper "Language '
 'Models Can Teach Themselves to Use Tools" on page 3 involves several key '
 'components:\n'
 '\n'
 '1. Sampling API Calls:\n'
 '   - A prompt P(x) is used to encourage the language model (LM) to annotate '
 'an example x with API calls.\n'
 '   - The probability pM(z_n+1 | z_1, ..., z_n) is computed to determine the '
 'likelihood of the LM assigning a token z_n+1 as a continuation for the '
 'sequence z_1, ..., z_n.\n'
 '   - Up to k candidate positions for API calls are sampled based on the '
 'probability pi = pM(<API> | P(x); x_1:i-1).\n'
 '   - Positions with probabilities above a sampling threshold s are kept, and '
 'for each position i in the set I, up to m API calls are obtained by sampling '
 'from M given the sequence [P(x); x_1; ...; x_i-1; <API>] as a prefix.\n'
 '\n'
 '2. Executing API Calls:\n'
 '   - All API calls generated by the LM are executed to obtain the '
 'corresponding results, wh

In [40]:
output = agent_executor.invoke({"input": "recommend the future works of the paper 'Language Models Can Teach Themselves to Use Tools'"})
# TODO : we do not need paperID in this tool.



> Entering new AgentExecutor chain...

Invoking: `recommend_citation` with `{'query': 'Language Models Can Teach Themselves to Use Tools'}`


[{'paperId': '7d8905a1fd288068f12c8347caeabefd36d0dd6c', 'title': 'Gorilla: Large Language Model Connected with Massive APIs', 'abstract': "Large Language Models (LLMs) have seen an impressive wave of advances recently, with models now excelling in a variety of tasks, such as mathematical reasoning and program synthesis. However, their potential to effectively use tools via API calls remains unfulfilled. This is a challenging task even for today's state-of-the-art LLMs such as GPT-4, largely due to their inability to generate accurate input arguments and their tendency to hallucinate the wrong usage of an API call. We release Gorilla, a finetuned LLaMA-based model that surpasses the performance of GPT-4 on writing API calls. When combined with a document retriever, Gorilla demonstrates a strong capability to adapt to test-time document changes,

In [41]:
pprint(output['output'])

('Here are some recommended papers related to "Language Models Can Teach '
 'Themselves to Use Tools":\n'
 '\n'
 '1. **Gorilla: Large Language Model Connected with Massive APIs**\n'
 '   - Abstract: Large Language Models (LLMs) have seen significant '
 'advancements recently, excelling in various tasks. Gorilla, a finetuned '
 'LLaMA-based model, surpasses the performance of GPT-4 in writing API calls. '
 'It demonstrates a strong capability to adapt to test-time document changes '
 'and mitigate hallucination issues.\n'
 '   - Publication Date: May 24, 2023\n'
 '\n'
 '2. **Mind2Web: Towards a Generalist Agent for the Web**\n'
 '   - Abstract: Introduces Mind2Web, a dataset for developing generalist '
 'agents for the web. It provides diverse tasks from real-world websites and '
 'explores using large language models for building generalist web agents.\n'
 '   - Publication Date: June 9, 2023\n'
 '\n'
 '3. **ToolLLM: Facilitating Large Language Models to Master 16000+ Real-world '
 'AP

In [19]:
# TODO : Change Arxiv Search enging to Semantic Scholar 
# TODO : error handling when the tool called.
# Because below code doesn't work. with the difference of 'Gorilla: '
# output = agent_executor.invoke({"input": "summary the abstract of the paper 'Gorilla: Large Language Model Connected with Massive APIs'"})

output = agent_executor.invoke({"input": "summary the abstract of the paper 'Large Language Model Connected with Massive APIs'"})



> Entering new AgentExecutor chain...

Invoking: `loadpaper` with `{'query': 'Large Language Model Connected with Massive APIs', 'start_page': 1}`


### Searched Paper ###
Title: Gorilla: Large Language Model Connected with Massive APIs
Authors: Joseph E. Gonzalez
Published: 2023-05-24T16:48:11Z
ID: 2305.15334v1
Page limit reached at 4
Gorilla: Large Language Model Connected with Massive APIs Shishir G. Patil1∗Tianjun Zhang1,∗Xin Wang2Joseph E. Gonzalez1 1UC Berkeley2Microsoft Research sgp@berkeley.edu Abstract Large Language Models (LLMs) have seen an impressive wave of advances re- cently, with models now excelling in a variety of tasks, such as mathematical reasoning and program synthesis. However, their potential to effectively use tools via API calls remains unfulfilled. This is a challenging task even for today’s state-of- the-art LLMs such as GPT-4, largely due to their inability to generate accurate input arguments and their tendency to hallucinate the wrong usage of an API c

In [20]:
pprint(output['output'])

('The paper titled "Large Language Model Connected with Massive APIs" '
 'introduces Gorilla, a finetuned LLaMA-based model that surpasses the '
 'performance of GPT-4 in writing API calls. Gorilla, when combined with a '
 'document retriever, demonstrates the ability to adapt to test-time document '
 'changes, reducing hallucination errors commonly encountered with LLMs. The '
 'paper introduces APIBench, a dataset consisting of HuggingFace, TorchHub, '
 "and TensorHub APIs, to evaluate the model's ability accurately. Gorilla "
 'significantly outperforms GPT-4 in terms of API functionality accuracy and '
 'reduces hallucination errors. The paper emphasizes the importance of '
 'empowering LLMs to use tools via API calls to access vast knowledge bases '
 'and accomplish complex computational tasks effectively.')


In [42]:
output = agent_executor.invoke({"input": "recommend the reference works of the paper 'Language Models Can Teach Themselves to Use Tools'"})



> Entering new AgentExecutor chain...

Invoking: `recommend_reference` with `{'query': 'Language Models Can Teach Themselves to Use Tools'}`


[{'paperId': '90abbc2cf38462b954ae1b772fac9532e2ccd8b0', 'title': 'Language Models are Few-Shot Learners', 'abstract': "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something which current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches. Specifically, we train GPT-3, an autore

In [43]:
pprint(output['output'])

('Here are some recommended reference works related to the paper "Language '
 'Models Can Teach Themselves to Use Tools":\n'
 '\n'
 '1. **Paper Title:** Language Models are Few-Shot Learners\n'
 '   - **Abstract:** Recent work has demonstrated substantial gains on many '
 'NLP tasks and benchmarks by pre-training on a large corpus of text followed '
 'by fine-tuning on a specific task. This paper shows that scaling up language '
 'models greatly improves task-agnostic, few-shot performance, sometimes even '
 'reaching competitiveness with prior state-of-the-art fine-tuning '
 'approaches.\n'
 '   - **Publication Date:** 2020-05-28\n'
 '\n'
 '2. **Paper Title:** PaLM: Scaling Language Modeling with Pathways\n'
 '   - **Abstract:** This paper explores the impact of scale on few-shot '
 'learning by training a 540-billion parameter, densely activated, Transformer '
 'language model called Pathways Language Model PaLM. It achieves breakthrough '
 'performance on language understanding and 

### Todo : Make Agent with Llama

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [44]:
# we can use 262k version of Llama!
model_name = 'gradientai/Llama-3-8B-Instruct-262k'

dtype = "float16"
if torch.cuda.is_bf16_supported():
    dtype = "bfloat16"

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=getattr(torch, dtype), quantization_config=quantization_config, 
                                             device_map="auto",  cache_dir=os.getenv("HF_HOME", "~/.cache/huggingface"))
model.eval()

tokenizer = AutoTokenizer.from_pretrained(model_name)